In [1]:
import numpy as np
import pandas as pd 


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
clf = RandomForestClassifier(n_estimators = 100) 

from sklearn import metrics

def load (input):
    
    data = pd.read_csv(input, delimiter = ",")
    extraced_ssid_level_pivot =data.pivot(columns=['bssid'], values='level')
    
    output =pd.concat([data, extraced_ssid_level_pivot] , axis=1)
    if 'level' in output:
        del output['level']
    if 'ssid' in output:
        del output['ssid']
    if 'bssid' in output:
        del output['bssid']
    return output

def get_features_and_target(df , target_to_predicte):
    target=df[target_to_predicte]
    if target_to_predicte in df:
        del df[target_to_predicte]
    features=df
    return features, target

dataset  = load('donnees.txt') ;   # on souhaitera ajouter des donnees dans ce dataset
#dataset['salleid'] = dataset['salleid'].astype(str)


In [2]:

dataset = dataset.groupby('positionid').agg("max").fillna(100)
dataset

,batimentid,salleid,floorid,centrefrequence0,frequency,00:4e:35:c8:ce:24,1c:28:af:b2:95:a0,1c:28:af:b2:95:a1,1c:28:af:b2:95:a2,1c:28:af:b2:95:a3,...,9c:1c:12:1d:9e:40,9c:1c:12:1d:9e:41,9c:1c:12:1d:9e:42,9c:1c:12:1d:9e:43,9c:1c:12:1d:9e:45,bc:9f:e4:8b:52:a0,bc:9f:e4:8b:52:a2,bc:9f:e4:8b:52:a3,bc:9f:e4:8b:52:a4,bc:9f:e4:8b:52:a5
positionid,,,,,,,,,,,,,,,,,,,,,
1,1,9100,1,5745,5745,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-88.0,-74.0,-73.0,-73.0
2,1,9100,1,5785,5785,100.0,100.0,-86.0,-86.0,100.0,...,100.0,100.0,100.0,100.0,100.0,-71.0,-66.0,-69.0,-68.0,-70.0
3,1,9100,1,5785,5785,100.0,100.0,-84.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,-85.0,100.0
4,1,9100,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,-75.0,100.0,100.0
5,1,9100,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,100.0,100.0,100.0
6,1,9100,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,100.0,100.0,100.0
7,1,9100,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,100.0,100.0,100.0
8,1,404,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,-88.0,100.0,100.0,100.0,100.0,100.0
9,1,404,1,5785,5785,-90.0,100.0,100.0,100.0,100.0,...,-87.0,-86.0,-87.0,-88.0,100.0,100.0,100.0,100.0,100.0,100.0


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
def learn_and_generate_model() : 
    X, y =get_features_and_target(dataset, 'salleid')
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.50, random_state=42)
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X_train, np.asarray(y_train, dtype="|S6"))
    y_pred =  neigh.predict(X_test)
    # metrics are used to find accuracy or error   
    print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(np.asarray(y_test, dtype="|S6"), y_pred))
    return neigh , confusion_matrix(np.asarray(y_test, dtype="|S6"), y_pred)

neigh , conf = learn_and_generate_model()
print(conf)

ACCURACY OF THE MODEL:  0.9230769230769231
[[2 0 0 0]
 [0 2 0 0]
 [0 0 5 0]
 [0 0 1 3]]


In [4]:
neigh

KNeighborsClassifier(n_neighbors=3)

In [5]:
import pickle
from sklearn import tree
pickle.dump(clf, open('model.pkl','wb'))

In [6]:
#data = dataset['00:4e:35:c8:ce:24']=9999
def clean_input(dataset , X_dict): 
    X_dataframe = pd.DataFrame.from_dict(X_dict)
    for feature in X_dataframe: 
        if feature in dataset:
            print(feature)
        else:
            del X_dataframe[feature]
    return  X_dataframe

def threat_input(dataset , X_input):
    X_input= clean_input(dataset , X_input)
    key_array = []
    for feature in X_input:
        key_array.append(feature)
    values = X_input.values
    maj_dataset =  dataset.append(dict(zip(key_array, *values)), ignore_index=True)
    formated_input = maj_dataset.replace(np.nan , 100)
    formated_input = formated_input[-1:].values
    return formated_input
    
    

In [ ]:
from flask import Flask, request, redirect, url_for, flash, jsonify
import numpy as np
import pickle as p
import json
from pandas import json_normalize

app = Flask(__name__)

@app.route('/predict', methods=['GET','POST'])
def makecalc():
    print(request.json)
    observation = threat_input(dataset , request.json)
    print(threat_input(dataset , request.json))
    print ("predicted" , neigh.predict(observation) , type(neigh.predict(observation)))
    return np.array2string(neigh.predict(observation))

@app.route('/app', methods=['GET','POST'])
def makecalcc():
    print(request.json)
    return "computed"
if __name__ == '__main__':
    modelfile = 'model.pkl'
    model = p.load(open(modelfile, 'rb'))
    app.run(debug=False, host='0.0.0.0')
# ici le serveur traite les donnees qu'il recoit 


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [22/Apr/2021 14:28:34] "POST /predict HTTP/1.1" 200 -


{'1c:28:af:ce:e7:14': {'level': -36}, '1c:28:af:ce:e7:13': {'level': -35}, '1c:28:af:b2:f7:74': {'level': -83}, '1c:28:af:b2:f7:73': {'level': -83}, '1c:28:af:b2:f7:72': {'level': -83}, '1c:28:af:b2:f7:71': {'level': -84}, '1c:28:af:b2:f7:70': {'level': -83}, '1c:28:af:ce:e7:12': {'level': -36}, '1c:28:af:ce:e7:11': {'level': -36}, '1c:28:af:ce:e7:10': {'level': -36}, '1c:28:af:ce:1b:d2': {'level': -70}, '1c:28:af:ce:1b:d1': {'level': -70}, '1c:28:af:ce:1b:d0': {'level': -70}, '1c:28:af:b4:1c:e4': {'level': -74}, '1c:28:af:b4:1c:e1': {'level': -78}, '1c:28:af:ce:e7:04': {'level': -59}, '1c:28:af:ce:e7:03': {'level': -59}, '1c:28:af:ce:e7:02': {'level': -43}, '1c:28:af:ce:e7:01': {'level': -43}, '1c:28:af:ce:e7:00': {'level': -43}, 'd8:9e:61:cf:a2:76': {'level': -73}}
1c:28:af:ce:e7:14
1c:28:af:ce:e7:13
1c:28:af:b2:f7:72
1c:28:af:b2:f7:71
1c:28:af:b2:f7:70
1c:28:af:ce:e7:12
1c:28:af:ce:e7:11
1c:28:af:ce:e7:10
1c:28:af:ce:1b:d2
1c:28:af:ce:1b:d1
1c:28:af:ce:1b:d0
1c:28:af:b4:1c:e4
1c:28:

172.17.0.1 - - [22/Apr/2021 14:38:54] "POST /app HTTP/1.1" 200 -


{'1c:28:af:ce:e7:00': {'level': -43}, '1c:28:af:ce:e7:03': {'level': -43}, '1c:28:af:ce:e7:01': {'level': -43}, '1c:28:af:ce:e7:02': {'level': -43}, '1c:28:af:ce:e7:10': {'level': -44}, '1c:28:af:ce:e7:04': {'level': -44}, '1c:28:af:ce:e7:11': {'level': -44}, '1c:28:af:ce:e7:12': {'level': -46}, '1c:28:af:b2:f7:60': {'level': -65}, '1c:28:af:ce:1b:c2': {'level': -66}, 'd8:9e:61:cf:a2:76': {'level': -66}, '1c:28:af:b2:f7:61': {'level': -67}, '1c:28:af:ce:1b:c1': {'level': -67}, '1c:28:af:ce:1b:c3': {'level': -67}, '1c:28:af:b2:f7:62': {'level': -71}, '1c:28:af:ce:1b:c0': {'level': -72}, '1c:28:af:ce:1b:d2': {'level': -78}, '1c:28:af:ce:1b:d0': {'level': -79}, '1c:28:af:ce:1b:d1': {'level': -79}, '1c:28:af:b4:1c:e2': {'level': -81}, '1c:28:af:b4:1c:e0': {'level': -82}, '1c:28:af:b4:1c:e1': {'level': -82}, 'bc:9f:e4:8b:52:a2': {'level': -84}, '1c:28:af:cd:23:30': {'level': -84}, '1c:28:af:b2:f7:72': {'level': -85}, '1c:28:af:cd:23:31': {'level': -85}, '1c:28:af:cd:23:32': {'level': -85}, 

...

[[100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
  100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
  100. 100. 100. 100. 100. 100. -78. -77. -81. -83. -83. -58. -58. -58.
  -58. -58. 100. 100. 100. 100. 100. 100. 100. 100. -76. -78. -78. -78.
  -79. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
  100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. -74.
  100. -74. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. -72. -73.
  -71. -85. -86. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
  100. 100. -74. -74. 100. 100. 100. -69. -70. -70. 100. 100. 100. 100.
  100. 100. 100. 100. 100. 100. 100. 100. -86. -86. -86. 100. 100. -67.
  -66. -66. -67. -66. 100. 100. 100. 100. 100. -68. -69. -70. 100. -79.
  -79. -72. -73. 100. -71. -72. -71. -60. -60. -68. -68. -67. -70. -70.
  -62. -63. -63. -62. -64. 100. 100. 100. 100. 100. 100. 100. 100. 100.
  100. 100. -85. 100.]]
predicted [b'108'] <class 'numpy.ndarray

172.17.0.1 - - [22/Apr/2021 14:56:13] "POST /predict HTTP/1.1" 200 -


{'1c:28:af:ce:e7:14': {'level': -64}, '1c:28:af:ce:e7:13': {'level': -62}, '1c:28:af:ce:e7:12': {'level': -63}, '1c:28:af:ce:e7:11': {'level': -63}, '1c:28:af:ce:e7:10': {'level': -62}, '1c:28:af:b2:c8:54': {'level': -58}, '1c:28:af:b2:c8:53': {'level': -58}, '1c:28:af:b2:c8:51': {'level': -58}, '1c:28:af:b2:c8:50': {'level': -58}, '1c:28:af:ce:1b:d2': {'level': -71}, '1c:28:af:ce:1b:d1': {'level': -72}, '1c:28:af:ce:1b:d0': {'level': -71}, '1c:28:af:ce:e7:00': {'level': -68}, '1c:28:af:ce:e7:04': {'level': -70}, '1c:28:af:ce:e7:03': {'level': -70}, '1c:28:af:ce:e7:02': {'level': -67}, '1c:28:af:ce:e7:01': {'level': -68}, 'd8:9e:61:cf:a2:76': {'level': -54}, '1c:28:af:b2:f7:64': {'level': -79}, '1c:28:af:b2:f7:63': {'level': -78}, '1c:28:af:b2:f7:62': {'level': -78}, '1c:28:af:b2:c8:52': {'level': -58}, '1c:28:af:b4:1c:e1': {'level': -73}, '1c:28:af:b4:1c:e2': {'level': -71}, '1c:28:af:b4:1c:e0': {'level': -72}, '1c:28:af:b4:1c:e4': {'level': -86}, '1c:28:af:b4:1c:e3': {'level': -85}, 

172.17.0.1 - - [22/Apr/2021 14:56:19] "POST /predict HTTP/1.1" 200 -


{'1c:28:af:ce:e7:14': {'level': -64}, '1c:28:af:ce:e7:13': {'level': -62}, '1c:28:af:ce:e7:12': {'level': -63}, '1c:28:af:ce:e7:11': {'level': -63}, '1c:28:af:ce:e7:10': {'level': -62}, '1c:28:af:b2:c8:54': {'level': -58}, '1c:28:af:b2:c8:53': {'level': -58}, '1c:28:af:b2:c8:51': {'level': -58}, '1c:28:af:b2:c8:50': {'level': -58}, '1c:28:af:ce:1b:d2': {'level': -71}, '1c:28:af:ce:1b:d1': {'level': -72}, '1c:28:af:ce:1b:d0': {'level': -71}, '1c:28:af:ce:e7:00': {'level': -68}, '1c:28:af:ce:e7:04': {'level': -70}, '1c:28:af:ce:e7:03': {'level': -70}, '1c:28:af:ce:e7:02': {'level': -67}, '1c:28:af:ce:e7:01': {'level': -68}, 'd8:9e:61:cf:a2:76': {'level': -54}, '1c:28:af:b2:f7:64': {'level': -79}, '1c:28:af:b2:f7:63': {'level': -78}, '1c:28:af:b2:f7:62': {'level': -78}, '1c:28:af:b2:c8:52': {'level': -58}, '1c:28:af:b4:1c:e1': {'level': -73}, '1c:28:af:b4:1c:e2': {'level': -71}, '1c:28:af:b4:1c:e0': {'level': -72}, '1c:28:af:b4:1c:e4': {'level': -86}, '1c:28:af:b4:1c:e3': {'level': -85}, 

In [ ]:
#data = dataset['00:4e:35:c8:ce:24']=9999
def clean_input(dataset , X_dict): 
    X_dataframe = pd.DataFrame.from_dict(X_dict)
    for feature in X_dataframe: 
        if feature in dataset:
            print(feature)
        else:
            del X_dataframe[feature]
    return  X_dataframe

def threat_input(dataset , X_input):
    X_input= clean_input(dataset , X_input)
    key_array = []
    for feature in X_input:
        key_array.append(feature)
    values = X_input.values
    maj_dataset =  dataset.append(dict(zip(key_array, *values)), ignore_index=True)
    formated_input = maj_dataset.replace(np.nan , 100)
    formated_input = formated_input[-1:].values
    return formated_input
    
    

In [ ]:
X ={'1c:28:af:ce:e7:14': {'level': -40}, '1c:28:af:ce:e7:13': {'level': -42}, '1c:28:af:cd:23:31': {'level': -85}, '1c:28:af:cd:23:30': {'level': -87}, '1c:28:af:ce:e7:12': {'level': -42}, '1c:28:af:ce:e7:11': {'level': -40}, '1c:28:af:ce:e7:10': {'level': -40}, '1c:28:af:b2:c8:54': {'level': -75}, '1c:28:af:ce:1b:d2': {'level': -70}, '1c:28:af:ce:1b:d1': {'level': -70}, '1c:28:af:ce:1b:d0': {'level': -70}, '1c:28:af:b2:c8:44': {'level': -86}}

In [ ]:
observation = threat_input(dataset , X)

In [ ]:
neigh.predict(observation)

In [ ]:
    print(request.json)
    majdataset = dataset.append(request.json, ignore_index=True)
    majdataset = majdataset.fillna(100)
    last_scan = majdataset[-1:].values
    salle_predicted = model.predict(last_scan)
    salle_predicted_json = np.array2string((salle_predicted))